<a href="https://colab.research.google.com/github/tarabelo/PIAC-2526/blob/main/Circuitos%20parametrizados%20y%20algoritmos%20variacionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalamos qiskit en el notebook
!pip install qiskit

# **Circuitos parametrizados y algoritmos variacionales**

### Contenidos

1. [Algoritmos cuánticos variacionales](#variacionales)
1. [Circuitos parametrizados en Qiskit](#parametros)

<a name="variacionales"></a>

## **Algoritmos cuánticos variacionales**
Problema de los computadores cuánticos actuales: el ruído destruye la superposición de estado.

No es posible resolver un problema complejo (profundo) en un sistema actual: por ejemplo, la factorización de Shor podría necesitar millones de cúbits para corrección de errores.

Computadores actuales: _Noisy intermediate-scale quantum_ (NISQ)

Podemos pensar en dividir un problema complejo en partes más simples, y utilizar CPUs tradicionales en el proceso de optimización:

<center><img src="https://drive.google.com/uc?export=view&id=1WgpYw-cCkpuNZE2AOnPL4b3kIt9A_kU2" alt="Algoritmo híbrido" width="700"  /></center>

Esta estrategia también se denomina _Computación híbrida_.

Se basa en usar un circuito cuántico que incluye parámetros que se van optimizando. Este circuitos se  denomina *ansatz*, _forma (o circuito) variacional_ o, simplemente, circuito cuántico parametrizado.



---

---





<a name="parametros"></a>

## **Circuitos parametrizados en Qiskit**

Qiskit incluye métodos para facilitar la creación de circuitos parametrizados.

In [ ]:
from qiskit.circuit import Parameter, ParameterVector

#Se crean los parámetros usando un string como identificador (no se les asignan ningún valor)
parameter_0 = Parameter('θ')
parameter_1 = Parameter('β')
circuit = QuantumCircuit(1)

#Podemos ahora usar los parámetros como ángulos de rotación en una puerta
circuit.ry(theta = parameter_0, qubit = 0)
circuit.rx(theta = parameter_1, qubit = 0)

circuit.draw('mpl')

Es posible definir vectores de parámetros

In [ ]:
# Numero de cubits y de parametros
n = 2
num_par = 2*n

# ParameterVectors se inicializan con un identificador y un entero que indica la longitud del string
parameters = ParameterVector('θ', num_par)

circuit = QuantumCircuit(n)

for i in range(n):
    circuit.ry(parameters[i], i)
    circuit.rx(parameters[i+n], i)

circuit.draw('mpl')

Finalmente, se dan valores a los parámetros.

In [ ]:
# Creamos un diccionario que asigna a cada parámetro un valor aleatorio
param_dict = {p: np.random.random() for p in parameters}
print(param_dict)

# El método assign_parameters permite ligar los valores al circuito
circuit_v = circuit.assign_parameters(parameters = param_dict)
circuit_v.draw('mpl')

-------------------

## Ejercicio

Crea el siguiente circuito parametrizado, que alterna capas de rotaciones $R_y$ con CNOTS. Esta forma variacional se denomina [RealAmplitudes](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RealAmplitudes.html) y se usa en química computacional y en problemas de clasificación en quantum machine learning.

<center><img src="https://drive.google.com/uc?export=view&id=1HKVx-jEzWhNPH-VNEpzATRyv0xG0fFfZ" alt="Ansatz Real Amplitudes" width="700"  /></center>


In [ ]:
from qiskit.circuit import Parameter, ParameterVector

# Número de cubits y capas Ry+CNOT (no cuenta la capa Ry final)
n = 4
num_layers = 2

# Crea el vector de parámetros
parameters = ParameterVector('t', n*(num_layers+1))

# Crea el circuito
circuit = QuantumCircuit(n)

# Crea las capas Ry+CNOT
for layer in range(num_layers):
    #
    # TODO: Añade las puertas Ry parametrizadas usando el vector de parámetros para obtener el estado 𝜓0
    #
    for i in range(n):
      circuit.ry(parameters[i+layer*n],i)


    # Barrera para visualización
    circuit.barrier()

    #
    # TODO: Añade las puertas CNOT
    #
    for i in range(n-1,0,-1):
      circuit.cx(i-1,i)


    # Barrera para visualización
    circuit.barrier()

#
# TODO: Añade la capa Ry final
#
for i in range(n):
  circuit.ry(parameters[i+(layer+1)*n],i)

circuit.draw('mpl')

In [ ]:
# Este circuito está implementado en la librería de Qiskit
from qiskit.circuit.library import RealAmplitudes
n = 4
num_layers = 2
ansatz = RealAmplitudes(n, reps=num_layers, insert_barriers=True)
ansatz.decompose().draw('mpl')